In [1]:
use_jax = True

if use_jax:
    import jax.numpy as np
    from jax import jit
else:
    import numpy as np
    def jit(fun):
        return fun

def flatten(data, on_bad_data="warn"):
    """
    Flatten most data structures.
   
    Parameters
    ----------
    `data` : any
        The data structure to be flattened. Can also be a primitive.
    `on_bad_data` : `{"error", "ignore", "warn"}`, optional
        Specifies what to do when the data cannot be coerced to an
        ndarray. Options are as follows:
            - `"error"` : Raises a TypeError if data cannot be coerced
            to an ndarray.
            - `"ignore"` : Returns a list or, failing that, the original
            object if data cannot be coerced to an ndarray.
            - `"warn"` : Returns as in `"ignore"`, but raises a
            warning if data were not coerced to an ndarray.
        Defaults to `"warn"`.
   
    Returns
    -------
    `flattened_array` : array, list, or primitive
        Flattened version of `data`. If `on_bad_data="error"`, always
        an ndarray.
   
    """
    try:
        return np.ravel(np.asarray(data))
    except Exception:
        if isiterable(data):
            flattened_data = []
            if hasattr(data, "iteritems"):
                iterator = data.iteritems()
            else:
                iterator = enumerate(data)
            for __, datum in iterator:
                if isiterable(datum):
                    flattened_data.extend(flatten(datum))
                else:
                    flattened_data.append(datum)
        else:
            flattened_data = data
        try:
            return np.asarray(flattened_data)
        except Exception:
            if on_bad_data == "error":
                raise TypeError(_error_text(
                    [type(data), type(flattened_data)], "coercion"))
            elif on_bad_data == "ignore":
                return flattened_data
            else:
                warnings.warn(_error_text(
                    [type(data), type(flattened_data)], "coercion"), Warning)
                return flattened_data

            
def gmnd(data):
    """
    Geometric meandian, per xkcd.

    Parameters
    ----------
    `data` : numeric or array-like
        The data for which to take the measure of central tendency.
   
    Returns
    -------
    `central_tendencies` : dict of str -> float
        The measures of central tendency, ordered by their distance
        from the geometric meandian. Its keys are:
            - `"gmnd"` : geometric meandian (always first)
            - `"arithmetic"` : arithmetic mean
            - `"geometric"` : geometric mean
            - `"median"` : median
   
    """
    flat_data = flatten(data)
    mean_ = np.nanmean(flat_data)
    geomean_ = np.exp(np.nanmean(np.log(flat_data[flat_data > 0])))
    median_ = np.nanmedian(flat_data)
    data_i = np.asarray((mean_, geomean_, median_))
    converged = False
    runs = 0
    while not converged:
        runs = runs + 1
        data_i, converged = _gmnd_f(data_i)
    gmnd_ = data_i[0]
    avgs = np.asarray([gmnd_, mean_, geomean_, median_])
    errors = abs(np.repeat(gmnd_, 4) - avgs)
    named_errors = sorted(zip(
        errors, avgs, ["gmnd", "arithmetic", "geometric", "median"]))
    central_tendencies = {ne[2] : ne[1] for ne in named_errors}
    return central_tendencies


@jit
def _gmnd_f(data_i):
    """
    Backend geometric meandian.

    Parameters
    ----------
    `data_i` : ndarray of length 3
        The current iteration's arithmetic mean, geometric mean, and
        median.
    `tol` : float
        Absolute tolerance: iterate until the three measures of central
        tendency are within `tol` of each other.
   
    Returns
    -------
    `data_it1` : ndarray of length 3
        The next iteration's arithmetic mean, geometric mean, and
        median.
   
    """
    mean_ = np.nanmean(data_i)
    geomean_ = np.exp(np.nanmean(np.log(data_i)))
    median_ = np.nanmedian(data_i)
    data_iplus1 = np.asarray((mean_, geomean_, median_))
    converged = np.all(data_iplus1 == data_i)
    return data_iplus1, converged



In [64]:
x = [1.33270828512803, -12, np.nan, 500, 1e7, -400, 0, 1.31987323240687, 1.30510473067863, 1.29461675180401, 1.26362021158483, 1.2406368858161, 1.19733552025089, 1.17691238452997, 1.1726978597377, 1.17154490835153, 1.16242674828013, 1.15740084194638, 1.12291266992363, 1.11895718841847, 1.11354808327173, 1.11058408161182, 1.1098079302911, 1.10666903809116, 1.08201698406896, 1.07188634142266, 1.04703740710382, 1.04484053148134, 1.03329984113013, 1.02695645641247, 1.01206345044842, 0.990871667544852, 0.955834609670196, 0.952806071341893, 0.932278489355494, 0.930344594014769, 0.929040530149432, 0.877325363799783, 0.876032999810064, 0.86872318650742, 0.852457645396614, 0.848914622578293, 0.828801390416476, 0.826755710517616, 0.778298036068084, 0.775155388084465, 0.771939880511187, 0.757877042191858, 0.754645124457128, 0.740036242307919, 0.734381318998508, 0.710630684720229, 0.708680981933619, 0.686165804217841, 0.684662098252771, 0.684384932579798, 0.683698210858541, 0.665513189793097, 0.663081543508088, 0.656728510748847, 0.648087494329646, 0.632171377609169, 0.630905472334427, 0.629990706052419, 0.625344129019341, 0.605636352506457, 0.593089507369225, 0.588767545857126, 0.58873483462359, 0.584980921760774, 0.582006610962871, 0.566687845342609, 0.56391868053149, 0.562560648904584, 0.560783110392033, 0.541612292287957, 0.536608087438919, 0.535834535677185, 0.535790872488558, 0.5292353326372, 0.525410733337884, 0.51552031551905, 0.508402358017555, 0.506791867332958, 0.50650495024162, 0.502153258784974, 0.489023003960948, 0.48873275838628, 0.483387239190393, 0.478516456833015, 0.476790625149842, 0.462532489442999, 0.462268943985734, 0.461295461114494, 0.457506421066802, 0.453338084460388, 0.450557030048124, 0.448370721644599, 0.446462401473505, 0.440492854664286, 0.437445095423294, 0.43031794682178, 0.430058133988603, 0.428016045137849, 0.426573647438174, 0.410650014613906, 0.409717113097045, 0.403558656234613, 0.395051421298582, 0.391709460457939, 0.387566795136146, 0.38532261033173, 0.383391033544002, 0.380688348203487, 0.376842183525549, 0.374848681880602, 0.371107360721761, 0.370402709474014, 0.368824129312859, 0.367721435521303, 0.367477694999849, 0.363799369745165, 0.347870582538499, 0.345037874496436, 0.337883018483875, 0.336809099410001, 0.332813529552768, 0.332766350884797, 0.332327682414969, 0.332019429821481, 0.330287853472273, 0.327778535189989, 0.326618335137159, 0.326006771094425, 0.324553786531556, 0.319135690003935, 0.310344608300868, 0.30984463855816, 0.307574014271891, 0.305027870654707, 0.304558896102005, 0.303756669662139, 0.303575081336324, 0.299541312865505, 0.295949924297955, 0.292339120608975, 0.291963966123101, 0.290218206955022, 0.287360814743795, 0.286623049108003, 0.283426549259219, 0.278563455116399, 0.271973576202855, 0.270333282303014, 0.260701951383285, 0.25891545172287, 0.256047350188976, 0.255016950047918, 0.252853697446974, 0.242479461024035, 0.241335567873472, 0.238455992284198, 0.237380557943647, 0.232746623706951, 0.231858632138424, 0.223844683352535, 0.218099123416463, 0.216457684889442, 0.210261874420368, 0.20300386556335, 0.201540019390665, 0.200690410805314, 0.19528766740479, 0.191752771517835, 0.191018120764958, 0.186940456170616, 0.185790778798494, 0.18528619273415, 0.183761897791638, 0.183750559743082, 0.181476448683117, 0.179096150459403, 0.176441676778628, 0.174397400678615, 0.17296160729197, 0.17144736078297, 0.16804269704311, 0.166651573504735, 0.165279703056106, 0.160501645109946, 0.158027494137026, 0.15787569372432, 0.157629650865499, 0.155697633202443, 0.154877373724258, 0.154170270629412, 0.15228651572016, 0.151956364230164, 0.150731151648985, 0.148661607253246, 0.148143174292223, 0.146771406415153, 0.144146796102086, 0.142186108143445, 0.137402895017785, 0.135175156823796, 0.134435843556603, 0.13122729273972, 0.130031300840336, 0.128412812353246, 0.128327436663911, 0.128157917013693, 0.127481856838261, 0.121535768851209, 0.112697235953959, 0.111616983702258, 0.111381088864309, 0.110319962327951, 0.109247259376461, 0.107767491792566, 0.107382032884904, 0.104723221780301, 0.104586668117674, 0.104391342604245, 0.104340507874611, 0.10194425148774, 0.0994975104210725, 0.0988816414403261, 0.097357731651364, 0.0972355927051023, 0.0962208746665032, 0.095542220538335, 0.0892099533530437, 0.0886970700719415, 0.0877815210503383, 0.0871892756084974, 0.0840543721453662, 0.0814424537919145, 0.0801871587578697, 0.0798450577388009, 0.0778953144144452, 0.0771555213243459, 0.0754584609779819, 0.0745135582256619, 0.0713876945349385, 0.0697834496772614, 0.0697663830555743, 0.0697600894136252, 0.0695287227354028, 0.0694683184798301, 0.0687979534371116, 0.0682941127569752, 0.0681013184527879, 0.0668030397417894, 0.0666204877060974, 0.0663572321942544, 0.0659271104461369, 0.0625918965943423, 0.0610964632880338, 0.059334876262166, 0.0570609395184133, 0.0568173141884906, 0.0549416388568431, 0.0549121870943923, 0.0535325585063521, 0.0533729946903868, 0.0531852249682156, 0.0515242140015167, 0.0512285117759297, 0.0501288027259618, 0.0494092309335921, 0.0478862904956523, 0.0472989157492513, 0.0454533076924233, 0.0433241963652295, 0.043275020320379, 0.0428771009195106, 0.0413449550859487, 0.0410892549912814, 0.0410415354314893, 0.0404337822353524, 0.0398747611188535, 0.037848225473017, 0.036959085092766, 0.0368571879483787, 0.0314094111907425, 0.0288188798251758, 0.0283061347993513, 0.0281190336370403, 0.0275849124742807, 0.0249994231123883, 0.023052701719941, 0.0215689017911275, 0.019891919989414, 0.0180324971947874, 0.0178276061222888, 0.0177259114870081, 0.0173879062328509, 0.0152324955096375, 0.0150922189074331, 0.0143912432425389, 0.0138793711232246, 0.0136136104336424, 0.0131171692403783, 0.0117136158452005, 0.0104533280938683, 0.0103391937874655, 0.00953896123750761, 0.0091250686886907, 0.00859401929040825, 0.00792669174402632, 0.00541707936892629, 0.00495100880959116, 0.00250299764392558, 0.00196940242357138, 0.00189140809205869]
xa = np.asarray(x)
xm = np.asarray([value for value in gmnd(xa).values()])[1:4]

_gmnd_f(xm)

(DeviceArray([1.0582364e+04, 1.1561564e+01, 2.3845600e-01], dtype=float32),
 DeviceArray(False, dtype=bool))

In [65]:
gmnd(xa)

{'gmnd': 196.00555,
 'median': 0.238456,
 'geometric': 0.20414728,
 'arithmetic': 31746.648}

In [50]:
np.log(0)

DeviceArray(-inf, dtype=float32)

In [3]:
np.log(1-"f")

TypeError: unsupported operand type(s) for -: 'int' and 'str'

In [4]:
import warnings

In [10]:
import sys

In [13]:
type(sys.path[0])

str